In [12]:
# install plotly and dash, if not yet already
! pip install plotly dash

import plotly, dash
print(plotly.__version__)
print(dash.__version__)


5.9.0
2.17.1


In [31]:
import pandas as pd
from dash import Dash, html, dcc, Input, Output
import plotly.express as px

# Load Gyroscope data
df = pd.read_csv('cleaned_gyroscope_data.csv')

# Initialize Dash app with Bootstrap for styling
app = Dash(__name__, external_stylesheets=['https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css'])


app.layout = html.Div([
    html.H1("Gyroscope Data Visualization"),
    
    # Dropdown for selecting graph type
    html.Label("Select Graph Type:"),
    dcc.Dropdown(
        id='graph-type',
        options=[
            {'label': 'Line Chart', 'value': 'line'},
            {'label': 'Scatter Plot', 'value': 'scatter'},
            {'label': 'Distribution Plot', 'value': 'hist'}
        ],
        value='line'
    ),
    
    # Dropdown for selecting variables (X, Y, Z)
    html.Label("Select Data Variables:"),
    dcc.Dropdown(
        id='variable-selection',
        options=[
            {'label': 'X', 'value': 'x'},
            {'label': 'Y', 'value': 'y'},
            {'label': 'Z', 'value': 'z'},
            {'label': 'All', 'value': 'all'}
        ],
        value='all',
        multi=True
    ),
    
    # Textbox to select number of samples to display
    html.Label("Number of Samples to Display:"),
    dcc.Input(id='sample-count', type='number', value=100),
    
    # Graph to display gyroscope data
    dcc.Graph(id='gyro-graph'),
    
    # Previous/Next Buttons for navigation
    html.Button('Previous', id='previous-button', n_clicks=0),
    html.Button('Next', id='next-button', n_clicks=0),
    
    # Summary table of the data
    html.Div(id='data-summary')
])

# Define callbacks 
@app.callback(
    [Output('gyro-graph', 'figure'), Output('data-summary', 'children')],
    Input('graph-type', 'value'),
    Input('variable-selection', 'value'),
    Input('sample-count', 'value'),
    Input('previous-button', 'n_clicks'),
    Input('next-button', 'n_clicks')
)
def update_graph(graph_type, selected_vars, num_samples, prev_clicks, next_clicks):
    if num_samples is None:
        num_samples = 100

    # Handle navigation for data slices
    start_index = max(prev_clicks * num_samples, 0)
    end_index = min(start_index + num_samples, len(df))

    # Filter the data for the selected number of samples
    data_to_display = df.iloc[start_index:end_index]

    # Map 'x', 'y', 'z' to actual column names
    var_mapping = {'x': 'gyroX', 'y': 'gyroY', 'z': 'gyroZ'}

    # Handle the 'all' selection: If 'all' is selected, use all variables
    if 'all' in selected_vars:
        selected_vars = ['gyroX', 'gyroY', 'gyroZ']
    else:
        # If individual variables are selected, map them to actual columns
        selected_vars = [var_mapping[var] for var in selected_vars if var in var_mapping]

    # Ensure there's data to display
    if len(data_to_display) == 0:
        return {}, "No data available"

    # Plotting based on selected graph type and variables
    if graph_type == 'line':
        fig = px.line(data_to_display, x='timestamp', y=selected_vars)
    elif graph_type == 'scatter':
        fig = px.scatter(data_to_display, x='timestamp', y=selected_vars)
    else:
        fig = px.histogram(data_to_display, x='timestamp', y=selected_vars)

    # Statistical summary of the data
    summary_stats = data_to_display[selected_vars].describe()

    # Create a proper HTML table for the summary stats with Bootstrap classes
    summary_table = html.Table([
        html.Thead(html.Tr([html.Th("Statistic")] + [html.Th(col) for col in selected_vars])),
        html.Tbody([
            html.Tr([html.Td("Mean")] + [html.Td(f"{summary_stats[col]['mean']:.2f}") for col in selected_vars]),
            html.Tr([html.Td("Std Dev")] + [html.Td(f"{summary_stats[col]['std']:.2f}") for col in selected_vars]),
            html.Tr([html.Td("Min")] + [html.Td(f"{summary_stats[col]['min']:.2f}") for col in selected_vars]),
            html.Tr([html.Td("Max")] + [html.Td(f"{summary_stats[col]['max']:.2f}") for col in selected_vars]),
        ])
    ], className="table table-striped")

    return fig, summary_table


# Run the app 
if __name__ == '__main__':
    app.run_server(debug=True, mode='external', port=8051)
